# Project 2: Convolutional Networks

Instructions and assignment dates can be found on [github link](https://exception1984.github.io/CS294Y-2024/). 
Your submission will be this notebook, with all the outputs embedded in the notebook.
We will grade only what we can see in the notebook.

**Note**: This project is open-ended, meaning there are multiple valid solutions. While there are existing implementations of various deep networks for this task available online, plagiarism will NOT be tolerated. Your code will be checked for similarity against online resources as well as submissions from other students. You are expected to justify every design decision during the project evaluation. Any instance of plagiarism will result in a grade of ZERO without prior warning. Throughout the notebook, you will be explicitly instructed when it is appropriate to look up information online.

**Submission note:** We grade the content in this notebook. Make sure outputs are present. You will package everything in a zip file and submit it with the following format:
f"P2_{LastName}_{FirstName}.V{version_number}.zip" <br>e.g. "P2_Smit_John_V1.zip" - check more details on the announcement

## Introduction

- Your task is to develop a model for single-input image segmentation. The input will be an RGB image, and the output should be a dense classification map, where each pixel represents the predicted probability of belonging to a particular class. Image segmentation is a fundamental computer vision task with numerous practical applications, such as scene understanding and perception.

- We have provided the necessary source code for evaluating your model in `./utils.py`.

- Goals:
    - Get familiar with image segmentation data processing **[20 points]**
    - Implement a [UNet](https://arxiv.org/abs/1505.04597) model that takes an RGB image as input and outputs a dense segmentation map **[30 points]**
    - Define an appropriate loss function **[10 points]**
    - Tune the model to achieve a total Intersection over Union (IoU) score of **0.50** or higher on the provided validation set **[35 points]**
    - Visualize the model prediction **[5 points]**


## Setup

In [ ]:
!pip install tqdm datasets timm

In [2]:
import torch
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import timm
from torchvision import transforms as T
from PIL import Image
from utils import Metrics
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

In [3]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

## Downloading Data & Check

In [4]:
from datasets import load_dataset
ds_voc = load_dataset("zhyever/cs294y_project2_seg")

In [ ]:
ds_voc

In [6]:
def show(imgs, name=None):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    # plt.figure(figsize=(20,20))
    plt.axis('off')
    for i, img in enumerate(imgs):
        img = img.detach()
        img = TF.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
    fig.show()

def show_example_data(dataset, split='train', num=7):
    print(dataset[split][0]['image'])
    ims = [torch.tensor(np.asarray(dataset[split][i]['image'].resize((256, 256)))).permute(2, 0, 1) for i in range(num)]
    masks = [torch.tensor(np.asarray(dataset[split][i]['mask'].resize((256, 256)))).permute(2, 0, 1) for i in range(num)]
    pairs = [torch.cat([im, mask], dim=1) for im, mask in zip(ims, masks)]
    pairs_grid = make_grid(pairs, nrow=num)
    show(pairs_grid)


In [ ]:
show_example_data(ds_voc, split='train')

In [ ]:
show_example_data(ds_voc, split='val')

## Data Preparation [20 points]

- [5 points] Now you have to implement the encoding of the data. We have provided the decoding of the data for an inversed example

Make sure that the labels are in the range `0..N-1`, where
N is the number of classes - 21 in our case. You can have one special label for unknown regions.

We provide the map of RGB to label for convenience in `get_pascal_color_palette()`. The map should be read as this - if a pixel has color `[0, 0, 0]`, it has label 0. If the color is `[128, 0, 0]`, the label is 1 and so on.

You need to use the palette information to complete the `encode_segmap` function

In [9]:
def get_pascal_color_palette():
    """Load the mapping that associates pascal classes with label colors
    Returns:
        np.ndarray with dimensions (21, 3)
    """
    return np.asarray([[0, 0, 0],
                       [128, 0, 0],
                       [0, 128, 0],
                       [128, 128, 0],
                       [0, 0, 128],
                       [128, 0, 128],
                       [0, 128, 128],
                       [128, 128, 128],
                       [64, 0, 0],
                       [192, 0, 0],
                       [64, 128, 0],
                       [192, 128, 0],
                       [64, 0, 128],
                       [192, 0, 128],
                       [64, 128, 128],
                       [192, 128, 128],
                       [0, 64, 0],
                       [128, 64, 0],
                       [0, 192, 0],
                       [128, 192, 0],
                       [0, 64, 128]])

def get_pascal_class_names():
    return ['Background',
            'Aeroplane',
            'Bicycle',
            'Bird',
            'Boat',
            'Bottle',
            'Bus',
            'Car',
            'Cat',
            'Chair',
            'Cow',
            'Diningtable',
            'Dog',
            'Horse',
            'Motorbike',
            'Person',
            'Pottedplant',
            'Sheep',
            'Sofa',
            'Train',
            'Tvmonitor']

def encode_segmap(mask, unk_label=255):
    """Encode segmentation label images as pascal classes
    Args:
        mask (np.ndarray or PIL.Image.Image): raw segmentation label image of dimension
          (M, N, 3), in which the Pascal classes are encoded as colours.
    Returns:
        (np.ndarray): class map with dimensions (M, N), where the value at
        a given location is the integer denoting the class index.
    """
    mask = np.array(mask)
    out_mask = np.ones_like(mask[..., 0]) * unk_label
    for i, color in enumerate(get_pascal_color_palette()):
        # TODO: encode the segmentation map
        # NOTE: you can also completely modify this function
        
    return out_mask

def decode_segmap(mask, unk_label=255):
    """Decode segmentation label prediction as RGB images
    Args:
        mask (torch.tensor): class map with dimensions (B, M, N), where the value at
        a given location is the integer denoting the class index.
    Returns:
        (np.ndarray): colored image of shape (BM, BN, 3)
    """
    if isinstance(mask, torch.Tensor):
        mask = mask.cpu().numpy()
    mask = mask.astype(int)

    out_mask = np.stack([np.ones_like(mask)] * 3, axis=-1) * unk_label
    temp_mask = np.stack([mask] * 3, axis=-1)
    for i, color in enumerate(get_pascal_color_palette()):
        out_mask = np.where(np.all(temp_mask == i, axis=-1, keepdims=True), out_mask * 0 + color, out_mask)

    out_mask[mask==unk_label]=unk_label
    return out_mask


### Task 2 [15 points]

- **[5 points]** Please implement a horizontal flip as the data augmentation strategy here.

- It would be fine to add more data augmentations to increase performance, but it does NOT count for extra points. It would be super easy to reach the goal performance without any other augmentation, so no worries.

-  **[5 points]** Answer the question: why don't we implement augmentations for the validation/test set?

-  **[5 points]** Answer the question: why do we use the NEAREST strategy to resize the mask? Can we use bilinear? What is the difference? Which strategy would be the correct one?

In [10]:
im_transforms = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

def transform_val_voc(batch):
    batch['image'] = [im_transforms(i.resize((256, 256))) for i in batch['image']]
    batch['mask'] = [torch.tensor(encode_segmap(m.resize((256, 256), Image.NEAREST))) for m in batch['mask']]
    return batch

def transform_train_voc(batch):
    batch['image'] = [im_transforms(i.resize((256, 256))) for i in batch['image']]
    batch['mask'] = [torch.tensor(encode_segmap(m.resize((256, 256), Image.NEAREST))) for m in batch['mask']]
    # TODO: implement a horizontal flip here
    # HINTS: the shape of the image and mask tensors is different
    # HINTS: batch['image'][i] = function(batch['image'][i])
    # HINTS: batch['mask'][i] = function(batch['mask'][i])
    return batch


In [11]:
ds_voc['train'].set_transform(transform_train_voc)
ds_voc['val'].set_transform(transform_val_voc)

In [12]:
def denorm_imagenet(x):
    mean, std = torch.tensor(IMAGENET_MEAN).to(x.device), torch.tensor(IMAGENET_STD).to(x.device)
    if x.ndim == 3:
        mean, std = mean[:, None, None], std[:, None, None]
    elif x.ndim == 4:
        mean, std = mean[None, :, None, None], std[None, :,  None, None]
    return x * std + mean

def show_example_data(dataset, split='train', num=7, subset=False):
    im_list = []
    mask_list = []
    for i in range(num):
        if subset:
            sample = dataset[i]
        else:
            sample = dataset[split][i]
        im_list.append(denorm_imagenet(sample['image']))
        mask_list.append(torch.tensor(decode_segmap(sample['mask'])).permute(2, 0, 1) / 255)

    pairs = [torch.cat([im, mask], dim=1) for im, mask in zip(im_list, mask_list)]
    pairs_grid = make_grid(pairs, nrow=num)
    show(pairs_grid, 'after_transform.png')

In [ ]:
show_example_data(ds_voc, split='val')

Run the `show_example_data(ds_voc, split='train')` for multiple times to check if the augmentation works!

In [ ]:
show_example_data(ds_voc, split='train')

In [ ]:
show_example_data(ds_voc, split='train')

**Remember to answer the question!**

## Getting started with timm

[`timm`](https://timm.fast.ai/) is a deep-learning library created by [Ross Wightman](https://twitter.com/wightmanr) and is a collection of SOTA computer vision models.

We will be using `timm` to load encoder backbones pretrained on ImageNet so we don't have to define encoder architecture) or start training from scrath


In [16]:
BACKBONE = 'resnet18'

In [ ]:
# timm example for loading any encoder architecture
import timm
import torch

model = timm.create_model(BACKBONE) # load resnet18 architecture
x = torch.randn(1, 3, 224, 224)  # create random input tensor
model(x).shape

In [ ]:
# you can load a pretrained model
model = timm.create_model(BACKBONE, pretrained=True) # load resnet18 architecture
x = torch.randn(1, 3, 224, 224)  # create random input tensor
model(x).shape

In [ ]:
# set features_only as True since we only need features instead of the final ImageNet class logits
model = timm.create_model(BACKBONE, pretrained=True, features_only=True)
x = torch.randn(1, 3, 224, 224)  # create random input tensor
features = model(x)
[f.shape for f in features]

In [ ]:
# check useful information
model.feature_info.channels()

In [ ]:
# check useful information
model.default_cfg

In [ ]:
# List al available models
avail_pretrained_models = timm.list_models(pretrained=True)

print(f"Number of available pretrained models: {len(avail_pretrained_models)}")
print(f"First 5 models: {avail_pretrained_models[:5]}")

**NOTE**
- We don't constrain the usage of encoder architectures in this project. You can use any kind of encoders. But it would still be super easy to reach the goal performance with a pretrained **resnet18**encoder.

- If you are using other encoders, please check the mean and std value listed in the `encoder.default_cfg`. Keep IMAGENET_MEAN and IMAGENET_STD defined at the very beginning position of this notebook aligned with them.

## Model [30 points]

In [23]:
class Unet(nn.Module):
    """
    TODO: Complete the docstring of the Unet class (description, parameters, returns, etc.)
    """

    def __init__(
            self,
            backbone='resnet18',
            num_classes=1,
            # TODO: Add any other relevant args you need
    ):
        super().__init__()
        # TODO: Complete the definition of the Unet class
        # Use timm to load the encoder backbone with imagenet pretrained weights
        # encoder **must** return a list of feature maps that will be used by the decoder
        self.encoder = None
        self.decoder = None

    def forward(self, x: torch.Tensor):
        # TODO: Complete the forward function
        x = None
        return x

In [24]:
model = Unet(BACKBONE, num_classes=21)

In [ ]:
# Always perform a sanity check on the models you define
x = torch.randn(1, 3, 256, 256)
out = model(x)
print("Input shape", x.shape)
print("Output shape", out.shape)

## Loss Function [10 points]

In [26]:
def loss_fn(pred_y, y):
    # TODO: implement the loss function (a simple cross-entropy loss would work well)
    pass

## Training + Evaluation [35 points]

- **[30 points]** Tune the hyperparameters and the architecture to achieve the target IOU 

In [ ]:
# epochs = 
# batch_size = 
# workers = 
# learning_rate =
# wd =

run_id = f'model_n{epochs}_bs{batch_size}_lr{learning_rate}_wd{wd}'; print('\n\nTraining', run_id)
save_path = run_id + '.pkl'

In [28]:
def init_optimizer(model, lr, wd):
    return optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)

def init_scheduler(optimizer, epochs, train_loader, cycle_momentum, div_factor, final_div_factor, pct_start=0.7):
    lrs = [l['lr'] for l in optimizer.param_groups]
    return optim.lr_scheduler.OneCycleLR(optimizer, lrs, epochs=epochs, steps_per_epoch=len(train_loader),
                                         cycle_momentum=cycle_momentum, div_factor=div_factor,
                                         base_momentum=0.85, max_momentum=0.95,  
                                         final_div_factor=final_div_factor, pct_start=pct_start, 
                                         three_phase=False)

In [29]:
train_loader = DataLoader(ds_voc['train'], batch_size=batch_size, num_workers=workers, shuffle=True)
test_loader = DataLoader(ds_voc['val'], batch_size=1, num_workers=workers, shuffle=False, sampler=None)

ITER_PER_EPOCH = len(train_loader)
TOTAL_STEPS = ITER_PER_EPOCH * epochs

optimizer = init_optimizer(model, learning_rate, wd=wd)
scheduler = init_scheduler(optimizer, epochs, train_loader, cycle_momentum=True, div_factor=25, final_div_factor=10000, pct_start=0.3)

In [30]:
model = model.cuda()

In [31]:
@torch.no_grad()
def validate(model, valid_loader, tqdm_flag=False, print_results=False):
    metrics = Metrics(len(get_pascal_class_names()), get_pascal_class_names())
    model.eval()
    metrics.reset()

    if tqdm_flag:
        for i, (sample) in tqdm(enumerate(valid_loader), total=len(valid_loader), desc='Validating'):
            x, y = sample['image'].float().cuda(), sample['mask'].numpy()[:, :, :]
            y_pred = model(x)
            y_pred = F.interpolate(y_pred, size=(y.shape[-2:]), mode='nearest')
            y_pred = torch.argmax(y_pred, dim=1).detach().cpu().numpy() # get the most likely prediction
            metrics.add_batch(y, y_pred)
    else:
        for i, sample in enumerate(valid_loader):
            x, y = sample['image'].float().cuda(), sample['mask'].numpy()[:, :, :]
            y_pred = model(x)
            y_pred = F.interpolate(y_pred, size=(y.shape[-2:]), mode='nearest')
            y_pred = torch.argmax(y_pred, dim=1).detach().cpu().numpy() # get the most likely prediction
            metrics.add_batch(y, y_pred)
            
    table = metrics.get_table()

    if print_results:
        print('\nValidation stats ', table)
        
    return table.loc['total']['IoU']

In [32]:
log_term_dict = dict()
log_term_dict['loss'] = []
log_term_dict['val_metric'] = []
log_term_dict['lr'] = []
loss_log_interval = 100

- I have removed one line of the code in the following block, which is the core of the whole training pipeline. Please add it back to make the whole training script complete.

In [ ]:
pbar = tqdm(total=TOTAL_STEPS, desc='Training')
for epoch in range(epochs):
    model.train()
    N = len(train_loader)
        
    for i, (sample) in enumerate(train_loader):
        # Load a batch and send it to GPU
        x = sample['image'].float().cuda()
        y = sample['mask'].float().cuda()

        # Forward pass: compute predicted y by passing x to the model.
        y_pred = model(x, double_upsample=True)

        y = y.squeeze(1).long()

        # Compute and print loss.
        loss = loss_fn(y_pred, y)

        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable
        # weights of the model).
        optimizer.zero_grad()

        ##
        ##
        # I have removed one line of the code here
        ##
        ##

        # Calling the step function on an Optimizer makes an update to its parameters
        optimizer.step()
        # learning rate scheduler step.
        scheduler.step()

        pbar.update(1)
        # Report progress. Add any extra logging info here
        current_lr = scheduler.get_last_lr()[0]
        momentum = optimizer.param_groups[0]['betas'][0]
        
        pbar.set_postfix({'epoch': f'{epoch+1}/{epochs}', 
                          'loss': loss.item(),
                          'lr': current_lr,
                          'momentum': momentum,
                          'epoch%': "{0:.1f}%".format(100*(i+1)/N)})

        if i % loss_log_interval == 0:
            log_term_dict['loss'].append(loss.item())
            log_term_dict['lr'].append(current_lr)
            
            
    # Validation after every x epoch
    if (epoch+1) % 1 == 0:
        log_term = validate(model, test_loader)
        log_term_dict['val_metric'].append(log_term)
        print("Epoch {}, total IoU: {}".format(epoch+1, log_term))

torch.save(model.state_dict(), save_path)
print('\nTraining done. Model saved ({}).'.format(save_path))

plt.figure()
plt.figure(figsize=(50, 20))
plt.subplot(3, 1, 2)
x = [i for i in range(len(log_term_dict['loss']))]
plt.plot(x, log_term_dict['loss'])
plt.title('Training Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.subplot(3, 1, 3)
x = [i for i in range(len(log_term_dict['val_metric']))]
plt.plot(x, log_term_dict['val_metric'])
plt.title('Validation Metric')
plt.xlabel('Epochs')
plt.ylabel('IoU')
plt.subplot(3, 1, 1)
x = [i for i in range(len(log_term_dict['lr']))]
plt.plot(x, log_term_dict['lr'])
plt.title('Learning Rate')
plt.xlabel('Iterations')
plt.ylabel('LR')
plt.savefig('training_log_plot_{}.png'.format(run_id))

- This training script will save the model as well as the logs in the root folder. Please have a check after each training and optimize your hyperparameters and model architecure based on them.
  
- **[5 points]** While training your model, refer to the three plots saved in the `./figures` folder. Based on the curves, analyze and explain what occurred during these experiments. Hint: The figure names indicate the learning rate used for each experiment, and this is the only variable that differs between them. 

**Remember to answer the question!**

## Evaluation

- Evaluate your best model here

- The validation process will print a table with tons of numbers for different classes and metrics. We focus on the total - IoU one

In [ ]:
model = Unet(BACKBONE, num_classes=21)
model.cuda()
model.load_state_dict(torch.load(save_path, map_location='cpu'))
target_metric = validate(model, test_loader, tqdm_flag=True, print_results=True)
print("Target metric: {}".format(target_metric))

## Visualization  [5 points]
- Use the `decode_segmap` function to visualize images and their Ground Truth and Predicted segmentation maps. The images must be from the validation set.


In [ ]:
# TODO: Implement visualization
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

test_loader = DataLoader(ds_voc['val'], batch_size=1, num_workers=1, shuffle=False, sampler=None)
loader = iter(test_loader)

sample = next(loader)
x = sample['image'].float().cuda()

model = Unet(BACKBONE, num_classes=21)
model.cuda()
model.load_state_dict(torch.load(save_path, map_location='cpu'))
y_pred, y = model(x), sample['mask']

# TODO: Implement visualization